In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from mtl_pytorch.layer_node import Conv2dNode, BN2dNode

from mtl_pytorch.trainer import Trainer
from data.heads.pixel2pixel import ASPPHeadNode

from data.dataloader.cityscapes_dataloader import CityScapes
from data.metrics.pixel2pixel_loss import CityScapesCriterions
from data.metrics.pixel2pixel_metrics import CityScapesMetrics

from mobilenetv2 import mobilenet_v2

from data.dataloader.nyuv2_dataloader import NYU_v2
from data.metrics.pixel2pixel_loss import NYUCriterions
from data.metrics.pixel2pixel_metrics import NYUMetrics

from data.dataloader.taskonomy_dataloader import Taskonomy
from data.metrics.pixel2pixel_loss import TaskonomyCriterions
from data.metrics.pixel2pixel_metrics import TaskonomyMetrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [5]:
dataroot = 'datasets/Cityscapes/'

headsDict = nn.ModuleDict()
trainDataloaderDict = {}
valDataloaderDict = {}
criterionDict = {}
metricDict = {}

tasks = ['segment_semantic', 'depth_zbuffer']
task_cls_num = {'segment_semantic': 19, 'depth_zbuffer': 1}
for task in tasks:
    headsDict[task] = ASPPHeadNode(1280, task_cls_num[task])

    # For model trainer
    dataset = CityScapes(dataroot, 'train', task, crop_h=224, crop_w=224)
    trainDataloaderDict[task] = DataLoader(dataset, 16, shuffle=True)

    dataset = CityScapes(dataroot, 'test', task)
    valDataloaderDict[task] = DataLoader(dataset, 16, shuffle=True)

    criterionDict[task] = CityScapesCriterions(task)
    metricDict[task] = CityScapesMetrics(task)

# Model

In [16]:
mtlmodel = mobilenet_v2(False, headsDict=headsDict)
mtlmodel = mtlmodel.embed_nas(model=mtlmodel)

In [17]:
for node in mtlmodel.modules():
    if isinstance(node, Conv2dNode):
        print(node)

Conv2dNode(
  (taskOp): ModuleDict(
    (segment_semantic): Conv2d(1280, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(6, 6), dilation=(6, 6))
    (depth_zbuffer): Conv2d(1280, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(6, 6), dilation=(6, 6))
  )
  (basicOp): Conv2d(1280, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(6, 6), dilation=(6, 6))
  (policy): ParameterDict(
      (segment_semantic): Parameter containing: [torch.FloatTensor of size 3]
      (depth_zbuffer): Parameter containing: [torch.FloatTensor of size 3]
  )
  (dsOp): ModuleDict(
    (segment_semantic): ModuleList(
      (0): Conv2d(1280, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LazyLayer()
    )
    (depth_zbuffer): ModuleList(
      (0): Conv2d(1280, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

# Train

In [7]:
checkpoint = 'checkpoint/'
trainer = Trainer(mtlmodel, trainDataloaderDict, valDataloaderDict, criterionDict, metricDict, 
                  print_iters=100, val_iters=500, save_num=1, policy_update_iters=100)

### pre_train

In [8]:
trainer.pre_train(iters=1, lr=0.0001, savePath=checkpoint+'Cityscapes/')

### alter_train

In [22]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1, 'policy':0.0005}
trainer.alter_train_with_reg(iters=1, policy_network_iters=(100,400), policy_lr=0.01, network_lr=0.0001,
                             loss_lambda=loss_lambda,
                             savePath=checkpoint+'Cityscapes/')

Conv2dNode(
  (taskOp): ModuleDict(
    (segment_semantic): Conv2d(1280, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(6, 6), dilation=(6, 6))
    (depth_zbuffer): Conv2d(1280, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(6, 6), dilation=(6, 6))
  )
  (basicOp): Conv2d(1280, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(6, 6), dilation=(6, 6))
  (policy): ParameterDict(
      (segment_semantic): Parameter containing: [torch.FloatTensor of size 3]
      (depth_zbuffer): Parameter containing: [torch.FloatTensor of size 3]
  )
  (dsOp): ModuleDict(
    (segment_semantic): ModuleList(
      (0): Conv2d(1280, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LazyLayer()
    )
    (depth_zbuffer): ModuleList(
      (0): Conv2d(1280, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

AttributeError: 'Conv2dNode' object has no attribute 'depth'

### sample policy from trained policy distribution and save

In [23]:
policy_list = {'segment_semantic': [], 'depth_zbuffer': []}
name_list = {'segment_semantic': [], 'depth_zbuffer': []}

In [25]:
for name, param in mtlmodel.named_parameters():
    if 'policy' in name and not torch.eq(param, torch.tensor([0., 0., 0.])).all():
        if 'segment_semantic' in name:
            policy_list['segment_semantic'].append(param.data.cpu().detach().numpy())
            name_list['segment_semantic'].append(name)
        elif 'depth_zbuffer' in name:
            policy_list['depth_zbuffer'].append(param.data.cpu().detach().numpy())
            name_list['depth_zbuffer'].append(name)

In [26]:
from torch.nn.functional import softmax
from collections import OrderedDict

sample_policy_dict = OrderedDict()
for task in tasks:
    for name, policy in zip(name_list[task], policy_list[task]):
        distribution = softmax(policy, axis=-1)
        distribution /= sum(distribution)
        choice = np.random.choice((0,1,2), p=distribution)
        if choice == 0:
            sample_policy_dict[name] = torch.tensor([1.0,0.0,0.0]).cuda()
        elif choice == 1:
            sample_policy_dict[name] = torch.tensor([0.0,1.0,0.0]).cuda()
        elif choice == 2:
            sample_policy_dict[name] = torch.tensor([0.0,0.0,1.0]).cuda()

In [28]:
from experiments_sample import sample_path

sample_state = {'state_dict': sample_policy_dict}
torch.save(sample_state, sample_path + 'sample_policy.model')

TypeError: Couldn't parse file content!

### post train from scratch

In [27]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1}
trainer.post_train(iters=1, lr=0.001,
                   decay_lr_freq=4000, decay_lr_rate=0.5,
                   loss_lambda=loss_lambda,
                   savePath=checkpoint+'Cityscapes/', reload='sample_policy.model')

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint/Cityscapes/sample_policy.model'

### get the validation results in the paper 

In [ ]:
mtlmodel.load_state_dict(torch.load('CityScapes.model'))
trainer.validate('mtl', hard=True) 

# Inference

In [ ]:
mtlmodel.load_state_dict(torch.load('CityScapes.model'))
output = mtlmodel(x, task='segment_semantic', hard=True)